# Import necessary packages

In [ ]:
#Cell 1

import pandas as pd
import numpy as np
import geopandas as gp
from shapely.geometry import Point, LineString, shape

# Import necessary shapefiles

In [ ]:
#Cell 2

#Shapefile 1 - Importing Project Information
projects = gp.read_file("C:/Users/Shapefiles/ProjectList.shp")

In [ ]:
#Cell 3

#Shapefile 2 - Importing dataset of all roadway segments 
segments_main = gp.read_file("C:/Users/SHAPEFILES/roadway_segments.shp")

In [ ]:
#Cell 4

#Shapefile 3 - Importing dataset of all intersections
intersections_main = gp.read_file("C:/Users/SHAPEFILES/Intersections.shp")

In [ ]:
#Cell 5

#Shapefile 4 - Importing dataset of all ramps
ramps_main = gp.read_file("C:/Users/SHAPEFILES/Ramps.shp")

# Spatial Join

In [ ]:
#Cell 6

#Spatial join 1 - between projects and roadway segments
Roadway = gp.sjoin(projects, segments_main, how="inner", op="intersects")
Roadway.head ()

In [ ]:
#Cell 7

#Spatial join 2 - between projects and intersections
Intersection = gp.sjoin(projects, intersections_main, how="inner", op="intersects")
Intersection.head ()

In [ ]:
#Cell 8

#Spatial join 3 - between projects and roadway ramps
Ramp = gp.sjoin(projects, ramps_main, how="inner", op="intersects")
Ramp.head ()

# Roadway segment data

In [ ]:
#Cell 9

#If the beginning milepoint is in the middle of a segment, it will get "S"
#If the ending milepoint is in the middle of a segment, it will get "E"
#If the beginiing and ending mileipoint both fall in between a segment, it will get "M"
#Otherwise it will get "0"

def start_end(x):
    if x.BMP_prj>x.BMP_seg and x.EMP_prj <x.EMP_seg:
        return "M"
    elif x.BMP_prj>x.BMP_seg and x.BMP_prj<=x.EMP_seg:
        return "S"
    elif x.EMP_prj>=x.BMP_seg and x.EMP_prj<x.EMP_seg:
        return "E"
    else:
        return "O"

In [ ]:
#Cell 10

#Creating new column for "S", "E", "M" and "O"
Roadway['StartEnd']=Roadway.apply (start_end, axis = 1)
Roadway.head (2)

In [ ]:
#Cell 11

Roadway['StartEnd'].value_counts ()

In [ ]:
#Cell 12
# NEW BMP

#Replacing segment beginning mile points with project's beginning mile points
#For "S"  or "M" >> new_BMP = BMP_prj
#Otherwise new_BMP = BMP (segment)

def new_BMP (x):
    if x.StartEnd=="S":
        A = x.BMP_prj
        return A
    elif x.StartEnd=="M":
        B = x.BMP_prj
        return B
    else:
        C = x.BMP_seg
        return C

In [ ]:
#Cell 13
# NEW EMP

#Replacing segment ending mile points with project's ending mile points
#For "E"  or "M" >> new_EMP = EMP_prj
#Otherwise new_EMP = EMP (segment)

def new_EMP (x):
    if x.StartEnd=="E":
        D = x.EMP_prj
        return D
    elif x.StartEnd=="M":
        E = x.EMP_prj
        return E
    else:
        F = x.EMP_seg
        return F

In [ ]:
#Cell 14

#Adding column for New_BMP, New_EMP and New_Length
Roadway['New_BMP']=Roadway.apply (new_BMP, axis = 1)
Roadway['New_EMP']=Roadway.apply (new_EMP, axis = 1)
Roadway['New_Length'] = Roadway['New_EMP'] - Roadway['New_BMP']
Roadway.head (5)

In [ ]:
#Cell 15

#Calculating the modified EB and EECalt for the "S","E" and "M" segments. The values will remain the same for the "O" segments.
#KAB
Roadway["New_EB_KAB"] = (Roadway["New_Length"]/Roadway["Length_seg"]) * Roadway ["EB_KAB"]
Roadway["New_EEC_KAB"] = (Roadway["New_Length"]/Roadway["Length_seg"]) * Roadway ["EEC_KAB"]
Roadway["New_EECalt_KAB"] = (Roadway["New_Length"]/Roadway["Length_seg"]) * Roadway ["EECalt_KAB"]
#CO
Roadway["New_EB_CO"] = (Roadway["New_Length"]/Roadway["Length_seg"]) * Roadway ["EB_CO"]
Roadway["New_EEC_CO"] = (Roadway["New_Length"]/Roadway["Length_seg"]) * Roadway ["EEC_CO"]


Roadway.head ()

In [ ]:
#Cell 16

# Keep the necessary columns: Project_ID, modified columns of EB_KAB, EECalt_KAB, EB_CO and EEC_CO
Roadway_2 = Roadway [['Project_ID',
 'New_EB_KAB',
 'New_EECalt_KAB',
 'New_EB_CO',
 'New_EEC_CO']]
Roadway_2.head ()

In [ ]:
#Cell 17

#"Groupby with sum" will sum up each column based on same project_ID
Roadway_grouped=Roadway_2.groupby('Project_ID').sum().reset_index() 
Roadway_grouped.head ()

In [ ]:
#Cell 18

#Renaming the columns with "_road" suffix
Roadway_grouped = Roadway_grouped.rename(columns={"New_EB_KAB": "EB_KAB_road", "New_EECalt_KAB": "EECalt_KAB_road", "New_EB_CO": "EB_CO_road", "New_EEC_CO": "EEC_CO_road"})
Roadway_grouped.head ()

# Intersection data

In [ ]:
#Cell 19

#Keep the necessary columns only
Intersection_2 = Intersection [['Project_ID',
 'EB_KAB',
 'EECalt_KAB',
 'EB_CO',
 'EEC_CO']]

In [ ]:
#Cell 20

#"Groupby with sum" will sum up each column based on same project_ID
Int_grouped=Intersection_2.groupby('Project_ID').sum().reset_index() 
Int_grouped.head ()

In [ ]:
#Cell 21

#Renaming the columns with "_int" suffix
Int_grouped = Int_grouped.rename(columns={"EB_KAB": "EB_KAB_int", "EECalt_KAB": "EECalt_KAB_int", "EB_CO": "EB_CO_int", "EEC_CO": "EEC_CO_int"})
Int_grouped.head ()

# Ramp data

In [ ]:
#Cell 22

#Keep the necessary columns only
Ramp_2 = Ramp [['Project_ID',
 'EB_KAB',
 'EECalt_KAB',
 'EB_CO',
 'EEC_CO']]

In [ ]:
#Cell 23

#"Groupby with sum" will sum up each column based on same project_ID
Ramp_grouped=Ramp_2.groupby('Project_ID').sum().reset_index() 
Ramp_grouped.head ()

In [ ]:
#Cell 24

#Renaming the columns with "_ramp" suffix
Ramp_grouped = Ramp_grouped.rename(columns={"EB_KAB": "EB_KAB_ramp", "EECalt_KAB": "EECalt_KAB_ramp", "EB_CO": "EB_CO_ramp", "EEC_CO": "EEC_CO_ramp"})
Ramp_grouped.head ()

# Merging datasets for final score calculation

In [ ]:
#Cell 25

ID = projects ['Project_ID']
ID.head ()

In [ ]:
#Cell 26

# Merge 1 >> Project_ID and Roadway
merge1 = pd.merge (ID,Roadway_grouped,left_on = "Project_ID",right_on = "Project_ID", how = "left")
merge1.head ()

In [ ]:
#Cell 27

# Merge 2 >> Merge 1 and Intersection
merge2 = pd.merge (merge1,Int_grouped,left_on = "Project_ID",right_on = "Project_ID", how = "left")
merge2.head ()

In [ ]:
#Cell 28

# Merge 3 >> Merge 3 and Ramps
merge3 = pd.merge (merge2,Ramp_grouped,left_on = "Project_ID",right_on = "Project_ID", how = "left")
merge3.head ()

In [ ]:
#Cell 29

#Filling Nan by 0
merge3 = merge3.fillna(0) 
merge3.head ()

In [ ]:
#Cell 30

#Summing up the EB and EECalt of all three compontents of a project
#KAB
merge3 ['EB_KAB'] = merge3 ['EB_KAB_road'] + merge3 ['EB_KAB_int'] + merge3 ['EB_KAB_ramp']
merge3 ['EECalt_KAB'] = merge3 ['EECalt_KAB_road'] + merge3 ['EECalt_KAB_int'] + merge3 ['EECalt_KAB_ramp']

#CO
merge3 ['EB_CO'] = merge3 ['EB_CO_road'] + merge3 ['EB_CO_int'] + merge3 ['EB_CO_ramp']
merge3 ['EEC_CO'] = merge3 ['EEC_CO_road'] + merge3 ['EEC_CO_int'] + merge3 ['EEC_CO_ramp']


merge3.head ()

In [ ]:
#Cell 31

#Importing csv for weight calculation for KAB and CO 

df = pd.read_csv ("crash number and cost.csv", encoding = 'latin-1')
df


In [ ]:
#Cell 32

#Weight calculation for KAB (a) and CO (b)

df ['Total cost'] = df ['Comprehensive cost'] * df ['Number of crashes']
df ['KABCO_INDICATOR'] = np.where(df['Severity'].isin(['K','A','B']), 'KAB', 'CO')

df_grouped=df.groupby('KABCO_INDICATOR').agg({'Total cost':'sum','Number of crashes':'sum'}).reset_index()
df_grouped['Weighted_Cost']=df_grouped['Total cost']/df_grouped['Number of crashes']
df_grouped['Total_Weighted_Cost'] = df_grouped['Weighted_Cost'].sum()
df_grouped['Ratio'] = df_grouped['Weighted_Cost']/df_grouped['Total_Weighted_Cost']

df_grouped=df_grouped[['KABCO_INDICATOR','Ratio']]
ratio_dict= dict(zip(df_grouped['KABCO_INDICATOR'],df_grouped['Ratio']))

a = round (ratio_dict['KAB'],2)
b= round (ratio_dict['CO'],2)

a,b

In [ ]:
#Cell 33

#Final Score Calculation
merge3 ['Final_Score'] = (a/2)*merge3 ['EB_KAB'] + (a/2)*merge3 ['EECalt_KAB'] + (b/2)*merge3 ['EB_CO'] + (b/2)*merge3 ['EEC_CO']
merge3.head (2)

In [ ]:
#Cell 34

#Sorting the scores from highest to lowest
FINAL_Score_sorted = merge3.sort_values(by= "Final_Score", ascending=False)
FINAL_Score_sorted.head ()

In [ ]:
#Cell 35

#Adding Rank
FINAL_Score_sorted["Rank"] = FINAL_Score_sorted["Final_Score"].rank(ascending=False)
FINAL_Score_sorted.head ()

In [ ]:
#Cell 36

# Saving the file as .csv
FINAL_Score_sorted.to_csv ("Final_Score_by_ProjectID.csv")